# Enterprise Log File Analysis: Chargebee Case Study
This notebook analyzes one year of server log data to diagnose crawl budget waste, identify spider traps, and explain indexing issues.

**Objective:** Identify why some pages are "Crawled - Currently Not Indexed" and optimize Googlebot's efficiency.

## Phase 1: Environment & Dependency Setup

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import socket
import gc
from datetime import datetime
from google.colab import auth
import gspread
from google.auth import default

# Mount Google Drive
drive.mount('/content/drive')

# Constants
LOGS_PATH = '/content/drive/MyDrive/Chargebee_Logs/'
OUTPUT_SHEET_NAME = 'Chargebee_Audit_Findings'

sns.set(style="whitegrid")

## Phase 2: Data Ingestion (Memory Optimized)
We process logs in chunks and filter for 'Googlebot' immediately to keep the memory footprint small.

In [ ]:
def ingest_logs(directory_path, chunk_size=50000):
    all_files = glob.glob(os.path.join(directory_path, "*.csv"))
    if not all_files:
        print(f"No CSV files found in {directory_path}")
        return pd.DataFrame()

    googlebot_data = []
    
    # Columns we actually need to save memory
    use_cols = ['_time', 'useragent', 'uri_path', 'uri_query', 'status', 'bytes_sent', 'clientip', 'method']

    for file in all_files:
        print(f"Processing {file}...")
        try:
            # Read in chunks to avoid OOM
            for chunk in pd.read_csv(file, usecols=use_cols, chunksize=chunk_size, low_memory=False):
                # Filter for Googlebot (case-insensitive)
                filtered_chunk = chunk[chunk['useragent'].str.contains("googlebot", case=False, na=False)].copy()
                googlebot_data.append(filtered_chunk)
                
                # Aggressive garbage collection
                del chunk
                gc.collect()
        except Exception as e:
            print(f"Error processing {file}: {e}")

    if not googlebot_data:
        return pd.DataFrame()

    df = pd.concat(googlebot_data, ignore_index=True)
    print(f"Ingestion complete. Total Googlebot rows: {len(df)}")
    return df

df_raw = ingest_logs(LOGS_PATH)

## Phase 3: Data Cleaning & IP Verification

In [ ]:
def verify_googlebot_ip(ip):
    """Performs Double-Reverse DNS lookup to verify Googlebot."""
    try:
        host = socket.gethostbyaddr(ip)[0]
        if not (host.endswith('.googlebot.com') or host.endswith('.google.com')):
            return False
        
        addr = socket.gethostbyname(host)
        return addr == ip
    except (socket.herror, socket.gaierror):
        return False

def clean_data(df):
    if df.empty:
        return df

    # 1. Timestamp Conversion
    df['_time'] = pd.to_datetime(df['_time'])
    df.set_index('_time', inplace=True)
    
    # 2. IP Verification (Sample check if dataset is huge, or all unique IPs)
    unique_ips = df['clientip'].unique()
    print(f"Verifying {len(unique_ips)} unique IPs...")
    ip_map = {ip: verify_googlebot_ip(ip) for ip in unique_ips}
    df['is_verified_bot'] = df['clientip'].map(ip_map)
    
    # 3. File Type Categorization
    def get_file_type(path):
        if pd.isna(path): return 'Other'
        ext = os.path.splitext(path)[1].lower()
        if ext in ['', '.html', '.htm']: return 'HTML'
        if ext == '.js': return 'JS'
        if ext == '.css': return 'CSS'
        if ext in ['.jpg', '.jpeg', '.png', '.gif', '.svg', '.webp']: return 'Image'
        if ext in ['.json', '.xml']: return 'Data'
        return 'Other'

    df['file_type'] = df['uri_path'].apply(get_file_type)
    
    # 4. URL Normalization
    df['full_url'] = df['uri_path'] + df['uri_query'].fillna('')
    
    return df

df = clean_data(df_raw)
if not df.empty:
    print(f"Verified Bots: {df['is_verified_bot'].sum()} / {len(df)}")

## Phase 4: Core Analysis Modules

### Module A: Crawl Volume & Frequency

In [ ]:
# Daily Hits
daily_hits = df.resample('D').size()

plt.figure(figsize=(12, 6))
daily_hits.plot(title="Daily Googlebot Requests")
plt.ylabel("Number of Hits")
plt.show()

### Module B: Status Code Health

In [ ]:
status_summary = df.groupby(['status', 'file_type']).size().unstack(fill_value=0)
print("Status Code Distribution by File Type:")
display(status_summary)

# Calculate 304 vs 200 for Resources
resources = df[df['file_type'].isin(['JS', 'CSS'])]
if not resources.empty:
    cache_check = resources.groupby('status').size()
    perc_304 = (cache_check.get(304, 0) / len(resources)) * 100
    print(f"\nCache Control Check: {perc_304:.2f}% of JS/CSS requests returned 304 Not Modified.")
    if perc_304 < 90:
        print("ALERT: Low 304 percentage suggests Cache Control Optimization Opportunity.")

### Module C: Spider Trap Detection

In [ ]:
# Group by clean path and count unique full URLs
spider_traps = df.groupby('uri_path')['full_url'].nunique().sort_values(ascending=False).reset_index()
spider_traps.columns = ['uri_path', 'unique_variations']

top_traps = spider_traps[spider_traps['unique_variations'] > 50]
print("Potential Spider Traps (> 50 variations):")
display(top_traps.head(10))

### Module D: Rendering Budget Impact

In [ ]:
budget_impact = df.groupby('file_type')['bytes_sent'].sum().reset_index()
budget_impact['GB'] = budget_impact['bytes_sent'] / (1024**3)

plt.figure(figsize=(8, 8))
plt.pie(budget_impact['bytes_sent'], labels=budget_impact['file_type'], autopct='%1.1f%%')
plt.title("Crawl Budget by Content Type (Bandwidth)")
plt.show()

## Phase 5: Additional Visualization

In [ ]:
# Top 10 Crawled Directories
df['directory'] = df['uri_path'].apply(lambda x: '/' + x.split('/')[1] if pd.notna(x) and len(x.split('/')) > 1 else '/')
top_dirs = df['directory'].value_counts().head(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_dirs.values, y=top_dirs.index, palette="viridis")
plt.title("Top 10 Most Crawled Directories")
plt.xlabel("Hits")
plt.show()

# Status Code Distribution
status_counts = df['status'].value_counts().sort_index()
colors = {200: 'green', 301: 'yellow', 302: 'yellow', 304: 'blue', 404: 'orange', 500: 'red', 503: 'red'}
mapped_colors = [colors.get(x, 'grey') for x in status_counts.index]

plt.figure(figsize=(10, 6))
status_counts.plot(kind='bar', color=mapped_colors)
plt.title("Status Code Distribution")
plt.xlabel("Status Code")
plt.ylabel("Count")
plt.show()

## Phase 6: Export to Google Sheets
Publishes the findings to a Google Sheet for reporting.

In [ ]:
def export_to_sheets(spider_traps_df, status_df):
    auth.authenticate_user()
    creds, _ = default()
    gc_sheet = gspread.authorize(creds)

    try:
        sh = gc_sheet.open(OUTPUT_SHEET_NAME)
    except gspread.SpreadsheetNotFound:
        sh = gc_sheet.create(OUTPUT_SHEET_NAME)

    timestamp = datetime.now().strftime("%m/%d %H:%M")
    
    # Export Spider Traps
    ws_trap = sh.add_worksheet(title=f"Spider Traps {timestamp}", rows=len(spider_traps_df) + 10, cols=20)
    ws_trap.update('A1', [spider_traps_df.columns.values.tolist()] + spider_traps_df.fillna('').values.tolist())
    
    # Export Status Summary
    ws_status = sh.add_worksheet(title=f"Status Summary {timestamp}", rows=len(status_df) + 10, cols=20)
    # Flatten the status_df for export
    status_flat = status_df.reset_index()
    ws_status.update('A1', [status_flat.columns.values.tolist()] + status_flat.fillna('').values.tolist())
    
    print(f"Results exported successfully! URL: {sh.url}")

export_to_sheets(top_traps, status_summary)